SNEWS2.0 White Paper Event Totals Table
=================================

This notebook takes a model from the SNEWPY library and generates a time-independent snowglobes fluence file from the data. It then runs it through snowglobes (must be installed) and returns the summative counts, a routine to distill output to total counts is also included.

In [82]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from snewpy.models import *
import os
from shutil import copyfile
from astropy.io import ascii

In [17]:
def OutputInSnowGlobesFormat(SN,savefile="SNOformat.dat",d=10.):
    '''
    Write to disk a SnowGlobes fluence file summed over the entire SNEWPY model \
    Parameters
    ----------
    SN : SupernovaModel
        Supernova model from the SNEWPY library.  
    savefile : string
        path to file for the snowglobes fluence file
    d : float
        distance (in kpc) to place the model for the fluence calculation
    '''
    d *= 1000.*3.086e+18 #convert from kpc to cm
    keV=1e3
    MeV=1e6
    GeV=1e9
    
    times=SN.get_time()
    dt = np.zeros(len(times))
    for i in range(len(times)-1):
        dt[i] = times[i+1]-times[i]
    dt[len(times)-1] = times[-1]-times[-2]
    
    nenergybins = 501
    E=np.linspace(0,100,nenergybins)*MeV
    
    finalOF=np.zeros(len(Flavor)*len(E)).reshape(len(Flavor),len(E))
    OscillatedFluence = {}
    OscillatedSpectra = SN.get_oscillatedspectra(times[0],E)
    for flavor in range(len(Flavor)):
        OscillatedFluence[flavor] = OscillatedSpectra[flavor]*dt[0]* 200.*keV  / (4.*np.pi*d**2)
    for i in range(1,len(times)):
        OscillatedSpectra = SN.get_oscillatedspectra(times[i],E)
        for flavor in range(len(Flavor)):
            OscillatedFluence[flavor] += OscillatedSpectra[flavor]*dt[i]* 200.*keV  / (4.*np.pi*d**2)
            
    file = open(savefile,"w")
    
    for i in range(0,nenergybins):
        file.write("{0:.4f}".format(E[i]/GeV))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_e][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x][i]/2.))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x][i]/2.))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_e_bar][i]))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x_bar][i]/2.))
        file.write("\t"+str(OscillatedFluence[Flavor.nu_x_bar][i]/2.)+"\n")

    file.close()

In [3]:
def get_SGresults(SGdata,SG_settings,targettype,mass,eventtype="total"):
    '''
    small parser for the output from the snowglobes routine: make_event_table.pl
    
    Parameters
    ==========
    SGdata : dict
        dictionary of output (after splitlines) from the make_event_table.pl routine from snowglobes
    SG_settings : dict
        dictionary of settings describing the SGdata
    targettype : string
        dictionary identifier of the results wanted
    mass : float
        mass to scale snowglobes output to for the results wanted
    eventtype : string
        discriminates between total events, elastic scattering events or neutral current events,
        there is currently no individual channel parsing, although this data is in SGdata
    '''
    if eventtype == 'es': 
        line = -3
    elif eventtype == 'nc': 
        line = -2
    elif eventtype == 'total': 
        line = -1
    else:
        print("Warning, eventtype unknown, returning total")
        line = -1
    return float(SGdata[targettype][line].split(":")[-1])*mass/SG_settings[targettype][2]
        

Generate Fluence Files
==================

Here we generate snowglobes fluence files for three models and under three oscillation scenarios


In [14]:
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s11.2c", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat.dat",d=10.)

snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = Bollig2016("../../models/Bollig_2016/s27.0c", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat.dat",d=10.)

snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", NoTransformation(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat.dat",d=10.)
snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", AdiabaticMSW_IMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat.dat",d=10.)
snmodel = OConnor2015("../../models/OConnor_2015/M1_neutrinos.dat", AdiabaticMSW_NMO(),eos="LS220")
OutputInSnowGlobesFormat(snmodel,savefile="OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat.dat",d=10.)

Execute SNOwGLoBES
==================
Here we run SNOwGLoBES to determine total event rates in default detectors.  We store the summed data in order to use it for arbitrary detectors in the following

In [42]:
SG_DIR="/Users/evanoc/research/snowglobes/"
SNEWPY_DIR="/Users/evanoc/research/supernova_models/"
filebase = {"OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat",
            "OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat",
            "OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat",
            "Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat",
            "Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat"}

SG_settings={}
SG_settings["WC"]        = ["water", "wc100kt30prct",100]
SG_settings["argon"]     = ["argon", "ar40kt",40]
SG_settings["lead"]      = ["lead", "halo2",1]
SG_settings["scint"]     = ["scint", "scint20kt",20]
SG_settings["nova_soup"] = ["nova_soup", "novaFD",14]
SG_settings['icecube']   = ["water", "icecube", 51600]

allSGdata = {}
for SGfile in filebase:
    print("Working on",SGfile,"...")
    SGdata = {}
    for type in SG_settings:
        material = SG_settings[type][0]
        detector = SG_settings[type][1]
        os.chdir(SNEWPY_DIR+"doc/nb")
        copyfile(SGfile+'.dat', SG_DIR+"fluxes/"+SGfile+'.dat')
        os.chdir(SG_DIR)
        command = "./supernova.pl "+SGfile+" "+material+" "+detector
        os.system(command)
        command = "./make_event_table.pl  "+SGfile+" "+material+" "+detector
        SGdata[type] = os.popen(command).read().splitlines()
        os.chdir(SNEWPY_DIR+"doc/nb")
        #print(SG_settings[type][2],"kt of ",type,":", SGdata[type][-1])
    allSGdata[SGfile] = SGdata
    #print("\n")

Working on OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat
Working on Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat
Working on OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat
Working on Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat
Working on Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat
Working on Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat
Working on OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat
Working on Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat
Working on Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat


Generate table for SNEWS2.0 white paper
=================================

This will need style changes, but allows for easy recomputing if models change

In [94]:
filebase = {0:"OConnor2015_s40WH07_LS220_summed_NoOsc_SNOformat",
            1:"OConnor2015_s40WH07_LS220_summed_ANMO_SNOformat",
            2:"OConnor2015_s40WH07_LS220_summed_AIMO_SNOformat",
            3:"Bollig2016_s11.2c_LS220_summed_NoOsc_SNOformat",
            4:"Bollig2016_s11.2c_LS220_summed_ANMO_SNOformat",
            5:"Bollig2016_s11.2c_LS220_summed_AIMO_SNOformat",
            6:"Bollig2016_s27.0c_LS220_summed_NoOsc_SNOformat",
            7:"Bollig2016_s27.0c_LS220_summed_ANMO_SNOformat",
            8:"Bollig2016_s27.0c_LS220_summed_AIMO_SNOformat"}

maps = {"H$_2$O/$\\bar{\\nu}_e$":"WC","Long String/$\\bar{\\nu}_e$":"icecube","C$_n$H$_{2n}$/$\\bar{\\nu}_e$":"scint","Lead/$\\nu_e$":"lead","Ar/$\\nu_e$":"argon"}


data = {}
data['Experiments'] = ['Super-Kamiokande','Hyper-Kamiokande','IceCube','KM3NeT','KamLAND','Borexino',
                       'JUNO','SNO+','NO{$\\nu$}A','HALO','HALO-1kT','DUNE','MicroBooN','SBND']
data['Type'] = ["H$_2$O/$\\bar{\\nu}_e$","H$_2$O/$\\bar{\\nu}_e$","Long String/$\\bar{\\nu}_e$",
                "Long String/$\\bar{\\nu}_e$",'C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                'C$_n$H$_{2n}$/$\\bar{\\nu}_e$','C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                'C$_n$H$_{2n}$/$\\bar{\\nu}_e$','C$_n$H$_{2n}$/$\\bar{\\nu}_e$',
                "Lead/$\\nu_e$","Lead/$\\nu_e$","Ar/$\\nu_e$","Ar/$\\nu_e$","Ar/$\\nu_e$"]
data['Mass'] = [32,220,51600,0,1,0.278,20,0.7,15,0.079,1,40,0.09,0.12]
data['Location'] = ["Japan","Japan","South Pole","Mediterranean","Japan","Italy","China",
                    "Canada","USA","Canada","Italy","USA","USA","USA"]
data['$11.2 M_\\odot$'] = []
data['$27.0 M_\\odot$'] = []
data['$40.0 M_\\odot$'] = []
for experiment in range(len(data['Experiments'])):
    mass = data['Mass'][experiment]
    dettype = maps[data['Type'][experiment]]
    
    counts_LCN = int(get_SGresults(allSGdata[filebase[4]],SG_settings,dettype,mass))
    counts_LCI = int(get_SGresults(allSGdata[filebase[5]],SG_settings,dettype,mass))
    counts_MCN = int(get_SGresults(allSGdata[filebase[7]],SG_settings,dettype,mass))
    counts_MCI = int(get_SGresults(allSGdata[filebase[8]],SG_settings,dettype,mass))
    counts_HCN = int(get_SGresults(allSGdata[filebase[1]],SG_settings,dettype,mass))
    counts_HCI = int(get_SGresults(allSGdata[filebase[2]],SG_settings,dettype,mass))
    data['$11.2 M_\\odot$'].append(str(counts_LCN)+"/"+str(counts_LCI))
    data['$27.0 M_\\odot$'].append(str(counts_MCN)+"/"+str(counts_MCI))
    data['$40.0 M_\\odot$'].append(str(counts_HCN)+"/"+str(counts_HCI))
    
    
    
ascii.write(data,Writer=ascii.Latex)

\begin{table}
\begin{tabular}{ccccccc}
Experiments & Type & Mass & Location & $11.2 M_\odot$ & $27.0 M_\odot$ & $40.0 M_\odot$ \\
Super-Kamiokande & H$_2$O/$\bar{\nu}_e$ & 32.0 & Japan & 4053/4068 & 7771/7544 & 7597/4904 \\
Hyper-Kamiokande & H$_2$O/$\bar{\nu}_e$ & 220.0 & Japan & 27867/27968 & 53432/51866 & 52232/33720 \\
IceCube & Long String/$\bar{\nu}_e$ & 51600.0 & South Pole & 321043/332369 & 662149/659029 & 818230/625678 \\
KM3NeT & Long String/$\bar{\nu}_e$ & 0.0 & Mediterranean & 0/0 & 0/0 & 0/0 \\
KamLAND & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 1.0 & Japan & 189/188 & 363/349 & 350/233 \\
Borexino & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 0.278 & Italy & 52/52 & 101/97 & 97/64 \\
JUNO & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 20.0 & China & 3798/3769 & 7276/6995 & 7018/4661 \\
SNO+ & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 0.7 & Canada & 132/131 & 254/244 & 245/163 \\
NO{$\nu$}A & C$_n$H$_{2n}$/$\bar{\nu}_e$ & 15.0 & USA & 2849/2827 & 5457/5246 & 5263/3496 \\
HALO & Lead/$\nu_e$ & 0.079 & Canada & 4/3 & 9/8 & 10/